In [ ]:
# Análise dos 10 Logradouros - VERSÃO RAW
# Base: Planilha RAW (data/raw)
# Estrutura: Data, Equipe, Logradouro, Período, Quantidade
# Períodos: 10h (Manhã) e 15h (Tarde)
# Período: Solicitado ao usuário

# %% [markdown]
# # 1. Configuração Inicial

# %%
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("✓ Bibliotecas importadas")
print(f"✓ Análise iniciada em: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")

# %% [markdown]
# # 2. Definir Período de Análise

# %%
print("\n" + "=" * 80)
print("DEFINIR PERÍODO DO RELATÓRIO")
print("=" * 80)

data_inicio_str = input("\nDigite a DATA INICIAL (dd/mm/aaaa): ")
data_fim_str = input("Digite a DATA FINAL (dd/mm/aaaa): ")

# Converter datas
try:
    data_inicio_dt = datetime.strptime(data_inicio_str, "%d/%m/%Y")
    data_fim_dt = datetime.strptime(data_fim_str, "%d/%m/%Y")
    
    # Converter para formato string usado nos filtros
    data_inicio = data_inicio_dt.strftime('%Y-%m-%d')
    data_fim = data_fim_dt.strftime('%Y-%m-%d')
    
    print(f"\n✓ Período selecionado: {data_inicio_dt.strftime('%d/%m/%Y')} a {data_fim_dt.strftime('%d/%m/%Y')}")
    
    # Calcular quantidade de dias
    dias_periodo = (data_fim_dt - data_inicio_dt).days + 1
    total_periodos = dias_periodo * 2  # 2 períodos por dia (Manhã e Tarde)
    
    print(f"\n{'=' * 80}")
    print(f"Período: {data_inicio_dt.strftime('%d/%m/%Y')} a {data_fim_dt.strftime('%d/%m/%Y')}")
    print(f"  • Dias: {dias_periodo}")
    print(f"  • Períodos por dia: 2 (Manhã e Tarde)")
    print(f"  • Total de períodos: {total_periodos}")
    print(f"{'=' * 80}")
    
except ValueError:
    print("❌ Formato de data inválido! Use dd/mm/aaaa")
    raise Exception("Formato de data inválido")

# %% [markdown]
# # 3. Selecionar Arquivo RAW

# %%
from pathlib import Path

print("\n" + "=" * 80)
print("SELEÇÃO DO ARQUIVO RAW")
print("=" * 80)

# Detectar raiz do projeto
script_dir = Path(__file__).parent if '__file__' in globals() else Path.cwd()
if script_dir.name == 'notebooks':
    project_root = script_dir.parent
elif script_dir.name == 'etl':
    project_root = script_dir.parent.parent
else:
    project_root = script_dir

pasta_raw = project_root / 'data' / 'raw'
print(f"📂 Pasta raw: {pasta_raw}")

# Listar arquivos disponíveis
arquivos_disponiveis = sorted(list(pasta_raw.glob('*.xlsx')), 
                               key=lambda x: x.stat().st_mtime, 
                               reverse=True)

if arquivos_disponiveis:
    print(f"\n📁 Arquivos disponíveis (mais recentes primeiro):")
    for i, arq in enumerate(arquivos_disponiveis, 1):
        modificado = datetime.fromtimestamp(arq.stat().st_mtime).strftime('%d/%m/%Y %H:%M')
        print(f"  {i}. {arq.name}")
        print(f"     Modificado em: {modificado}\n")
    
    print("=" * 80)
    selecao = int(input("Digite o número do arquivo que deseja analisar: "))
    arquivo_selecionado = arquivos_disponiveis[selecao - 1]
    print(f"✓ Arquivo selecionado: {arquivo_selecionado.name}")
else:
    print(f"\n⚠️ Nenhum arquivo .xlsx encontrado em '{pasta_raw}'")
    raise FileNotFoundError(f"Nenhum arquivo em {pasta_raw}")

print("=" * 80)

# %% [markdown]
# # 4. Carregar Dados RAW

# %%
print("\n" + "=" * 80)
print("CARREGANDO DADOS RAW")
print("=" * 80)

try:
    df = pd.read_excel(arquivo_selecionado)
    print(f"✓ Arquivo carregado: {arquivo_selecionado.name}")
    print(f"✓ Total de registros: {len(df):,}")
    print(f"\nColunas disponíveis:")
    for col in df.columns:
        print(f"  - {col}")
    
    # Verificar campos necessários
    campos_necessarios = ['Data', 'Logradouro', 'Período', 'Quantidade']
    campos_faltando = [c for c in campos_necessarios if c not in df.columns]
    
    if campos_faltando:
        print(f"\n✗ ERRO: Campos faltando: {campos_faltando}")
        raise KeyError("Campos necessários não encontrados")
    
except Exception as e:
    print(f"✗ ERRO ao carregar arquivo: {str(e)}")
    raise

# %% [markdown]
# # 5. Preparar Dados

# %%
print("\n" + "=" * 80)
print("PREPARANDO DADOS")
print("=" * 80)

# Converter data
df['Data'] = pd.to_datetime(df['Data'], errors='coerce')

# Garantir que Quantidade seja numérico
df['Quantidade'] = pd.to_numeric(df['Quantidade'], errors='coerce').fillna(0)

# Limpar espaços em branco no Logradouro e Período
df['Logradouro'] = df['Logradouro'].astype(str).str.strip()
df['Período'] = df['Período'].astype(str).str.strip()

print(f"✓ Campo 'Data' convertido para datetime")
print(f"✓ Campo 'Quantidade' convertido para numérico")
print(f"✓ Campo 'Logradouro' limpo")
print(f"✓ Registros preparados: {len(df):,}")

# Extrair região do logradouro (tudo antes do primeiro " - ")
df['Região'] = df['Logradouro'].str.split(' - ', n=1).str[0].str.strip()

print(f"\nRegiões encontradas:")
regioes_unicas = sorted(df['Região'].unique())
for regiao in regioes_unicas:
    qtd_log = df[df['Região'] == regiao]['Logradouro'].nunique()
    print(f"  - {regiao} ({qtd_log} logradouros)")

# Verificar períodos únicos ANTES do mapeamento
print(f"\nPeríodos encontrados (originais):")
for periodo in sorted(df['Período'].unique()):
    print(f"  - {periodo}")

# Mapear períodos: 10h → Manhã, 15h → Tarde
mapeamento_periodos = {
    '10h': 'Manhã',
    '15h': 'Tarde'
}

df['Período_padrao'] = df['Período'].map(mapeamento_periodos)

# Verificar se há períodos não mapeados
periodos_nao_mapeados = df[df['Período_padrao'].isna()]['Período'].unique()
if len(periodos_nao_mapeados) > 0:
    print(f"\n⚠️ ATENÇÃO: Períodos não mapeados encontrados:")
    for periodo in periodos_nao_mapeados:
        print(f"  - '{periodo}'")
    print(f"\n   Esses registros serão ignorados na análise.")
    # Remover registros não mapeados
    df = df[df['Período_padrao'].notna()].copy()

print(f"\nPeríodos padronizados:")
for periodo in sorted(df['Período_padrao'].unique()):
    print(f"  - {periodo}")

print(f"\n✓ Registros após limpeza: {len(df):,}")

# %% [markdown]
# # 6. Filtrar Período

# %%
print("\n" + "=" * 80)
print("FILTRANDO PERÍODO")
print("=" * 80)

# Filtrar período
df_periodo = df[
    (df['Data'] >= data_inicio) & 
    (df['Data'] <= data_fim)
].copy()

print(f"Data início: {data_inicio}")
print(f"Data fim: {data_fim}")
print(f"Dias no período: {dias_periodo}")
print(f"✓ Registros no período: {len(df_periodo):,}")
print(f"  Logradouros únicos: {df_periodo['Logradouro'].nunique()}")
print(f"  Regiões únicas: {df_periodo['Região'].nunique()}")
print(f"  Períodos únicos: {sorted(df_periodo['Período_padrao'].unique())}")

# %% [markdown]
# # 7. Calcular Soma Total por Região (para ordenação)

# %%
print("\n" + "=" * 80)
print("CALCULANDO SOMA TOTAL POR REGIÃO")
print("=" * 80)

# Calcular soma total de pessoas por região
soma_por_regiao = df_periodo.groupby('Região')['Quantidade'].sum().sort_values(ascending=False)

print(f"✓ Regiões ordenadas por soma total:")
for regiao, soma in soma_por_regiao.items():
    print(f"  {regiao}: {soma:,.0f} pessoas")

# Lista ordenada de regiões (será usada para ordenar as abas)
regioes_ordenadas = soma_por_regiao.index.tolist()

# %% [markdown]
# # 8. Agrupar por Região e Logradouro

# %%
print("\n" + "=" * 80)
print("AGRUPANDO POR REGIÃO E LOGRADOURO")
print("=" * 80)

# Agrupar por Região e Logradouro
df_agrupado = df_periodo.groupby(['Região', 'Logradouro']).agg({
    'Quantidade': 'sum'
}).reset_index()

# Renomear coluna
df_agrupado.rename(columns={'Quantidade': 'Soma pessoas'}, inplace=True)

# Calcular Média pessoas (soma / total de períodos)
df_agrupado['Média pessoas'] = df_agrupado['Soma pessoas'] / total_periodos

# Para cada logradouro, calcular média por período
print("\n✓ Calculando médias por período para cada logradouro...")

medias_lista = []

for idx, row in df_agrupado.iterrows():
    regiao = row['Região']
    logradouro = row['Logradouro']
    
    # Filtrar dados desse logradouro
    df_log = df_periodo[df_periodo['Logradouro'] == logradouro].copy()
    
    # Agrupar por período e somar
    soma_por_periodo = df_log.groupby('Período_padrao')['Quantidade'].sum()
    
    # Calcular média (dividir pela quantidade de dias)
    media_manha = soma_por_periodo.get('Manhã', 0) / dias_periodo
    media_tarde = soma_por_periodo.get('Tarde', 0) / dias_periodo
    
    medias_lista.append({
        'Região': regiao,
        'Logradouro': logradouro,
        'Soma pessoas': row['Soma pessoas'],
        'Média pessoas': row['Média pessoas'],
        'Manhã': media_manha,
        'Tarde': media_tarde
    })

df_completo = pd.DataFrame(medias_lista)

print(f"✓ Agrupamento concluído")
print(f"  Total de logradouros: {len(df_completo)}")
print(f"  Total de regiões: {df_completo['Região'].nunique()}")

# %% [markdown]
# # 9. Criar Rankings por Região (Média)

# %%
print("\n" + "=" * 80)
print("CRIANDO RANKINGS DE MÉDIA POR REGIÃO")
print("=" * 80)

rankings_media = {}

for regiao in regioes_ordenadas:
    df_regiao = df_completo[df_completo['Região'] == regiao].copy()
    
    # Ordenar por Média pessoas (decrescente)
    df_regiao = df_regiao.sort_values('Média pessoas', ascending=False)
    
    rankings_media[regiao] = df_regiao
    
    print(f"\n✓ Região: {regiao}")
    print(f"  Total de logradouros: {len(df_regiao)}")
    print(f"  Maior média: {df_regiao['Média pessoas'].max():.2f}")
    print(f"  Menor média: {df_regiao['Média pessoas'].min():.2f}")

# %% [markdown]
# # 10. Criar Abas no Excel

# %%
def limpar_nome_aba(nome):
    """Remove caracteres inválidos para nomes de abas do Excel"""
    # Caracteres inválidos no Excel: : \ / ? * [ ]
    caracteres_invalidos = [':', '\\', '/', '?', '*', '[', ']']
    nome_limpo = nome
    for char in caracteres_invalidos:
        nome_limpo = nome_limpo.replace(char, '-')
    # Limitar a 31 caracteres
    return nome_limpo[:31]

print("\n" + "=" * 80)
print("CRIANDO ABAS NO EXCEL POR REGIÃO")
print("=" * 80)

# Caminho do arquivo de saída
pasta_docs = project_root / 'docs'
pasta_docs.mkdir(parents=True, exist_ok=True)
output_file = pasta_docs / 'top10_raw_analise_por_regiao.xlsx'

# Criar o arquivo Excel
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    
    # Contador de abas para garantir nomes únicos
    nomes_abas_usados = {}
    
    # Para cada região (ordenadas por soma total)
    for regiao in regioes_ordenadas:
        
        print(f"\n{'=' * 80}")
        print(f"Processando Região: {regiao}")
        print(f"{'=' * 80}")
        
        # 1. ABA: Rank Média [Região]
        df_rank_media = rankings_media[regiao].copy()
        df_rank_media_exportar = df_rank_media[[
            'Logradouro', 
            'Soma pessoas', 
            'Média pessoas',
            'Manhã',
            'Tarde'
        ]].copy()
        df_rank_media_exportar.insert(0, '#', range(1, len(df_rank_media_exportar) + 1))
        
        # Nome da aba (limitar a 31 caracteres)
        nome_aba_media = f"Rank Média {regiao}"
        if len(nome_aba_media) > 31:
            nome_aba_media = f"Média {regiao}"[:31]
        
        # Limpar caracteres inválidos
        nome_aba_media = limpar_nome_aba(nome_aba_media)
        
        # Garantir nome único
        nome_aba_media_final = nome_aba_media
        contador = 1
        while nome_aba_media_final in nomes_abas_usados:
            nome_aba_media_final = f"{nome_aba_media[:28]}_{contador}"
            contador += 1
        nomes_abas_usados[nome_aba_media_final] = True
        
        df_rank_media_exportar.to_excel(writer, sheet_name=nome_aba_media_final, index=False)
        print(f"✓ Aba '{nome_aba_media_final}' criada ({len(df_rank_media_exportar)} logradouros)")
        
        # 2. ABAS INDIVIDUAIS: Para cada logradouro do Rank Média (NA ORDEM DO RANKING)
        logradouros_ordenados = df_rank_media['Logradouro'].tolist()
        
        for idx, logradouro in enumerate(logradouros_ordenados, 1):
            
            # Filtrar dados originais para esse logradouro
            df_detalhado = df_periodo[
                df_periodo['Logradouro'] == logradouro
            ].copy()
            
            # Criar pivot: transformar períodos em colunas
            df_pivot = df_detalhado.pivot_table(
                index='Data',
                columns='Período_padrao',
                values='Quantidade',
                aggfunc='sum',
                fill_value=0
            ).reset_index()
            
            # PREENCHER DATAS FALTANTES
            # Criar range completo de datas do período
            data_inicio_dt_range = pd.to_datetime(data_inicio)
            data_fim_dt_range = pd.to_datetime(data_fim)
            todas_datas = pd.date_range(start=data_inicio_dt_range, end=data_fim_dt_range, freq='D')
            
            # Criar DataFrame com todas as datas
            df_todas_datas = pd.DataFrame({'Data': todas_datas})
            
            # Fazer merge para incluir datas faltantes
            df_pivot = df_todas_datas.merge(df_pivot, on='Data', how='left')
            
            # Preencher valores faltantes com 0 (colunas de período)
            colunas_periodo = [col for col in df_pivot.columns if col != 'Data']
            df_pivot[colunas_periodo] = df_pivot[colunas_periodo].fillna(0)
            
            # Garantir que todas as colunas de período existam
            for periodo in ['Manhã', 'Tarde']:
                if periodo not in df_pivot.columns:
                    df_pivot[periodo] = 0
            
            # Preparar DataFrame para exportação na ordem correta
            df_exportar = pd.DataFrame()
            df_exportar['Logradouro'] = [logradouro] * len(df_pivot)
            df_exportar['Data'] = df_pivot['Data'].dt.strftime('%d/%m/%Y')
            df_exportar['Manhã'] = df_pivot['Manhã'].astype(int)
            df_exportar['Tarde'] = df_pivot['Tarde'].astype(int)
            
            # Nome da aba (limitado a 31 caracteres)
            # Extrair apenas o nome do logradouro (depois do " - ")
            if ' - ' in logradouro:
                nome_logr_curto = logradouro.split(' - ', 1)[1]
            else:
                nome_logr_curto = logradouro
            
            # Limpar caracteres inválidos
            nome_aba = limpar_nome_aba(nome_logr_curto)
            
            # Garantir nome único de aba
            nome_aba_final = nome_aba
            contador = 1
            while nome_aba_final in nomes_abas_usados:
                sufixo = f"_{contador}"
                nome_aba_final = nome_aba[:31-len(sufixo)] + sufixo
                contador += 1
            nomes_abas_usados[nome_aba_final] = True
            
            # Escrever aba
            df_exportar.to_excel(writer, sheet_name=nome_aba_final, index=False)
            print(f"  [{idx}/{len(logradouros_ordenados)}] ✓ '{nome_aba_final}' ({len(df_exportar)} dias)")

print(f"\n{'=' * 80}")
print(f"✓ Arquivo criado: {output_file}")
print(f"  Total de abas criadas: {len(nomes_abas_usados)}")
print(f"{'=' * 80}")

# %% [markdown]
# # 11. Resumo Executivo

# %%
print("\n" + "=" * 80)
print("RESUMO EXECUTIVO")
print("=" * 80)

# Contar total de logradouros
total_logradouros = df_completo['Logradouro'].nunique()
total_regioes = len(regioes_ordenadas)

# Resumo por região
resumo_regioes = []
for regiao in regioes_ordenadas:
    qtd_logradouros = len(rankings_media[regiao])
    soma_total = soma_por_regiao[regiao]
    
    resumo_regioes.append(f"""
  • {regiao}:
    - Logradouros: {qtd_logradouros}
    - Soma total: {soma_total:,.0f} pessoas""")

resumo_regioes_str = "\n".join(resumo_regioes)

print(f"""
ANÁLISE: RANKINGS POR REGIÃO (VERSÃO RAW)

ARQUIVO ANALISADO:
• {arquivo_selecionado.name}

PERÍODO:
• De: {data_inicio_dt.strftime('%d/%m/%Y')}
• Até: {data_fim_dt.strftime('%d/%m/%Y')}
• Total de dias: {dias_periodo}
• Total de períodos: {total_periodos} ({dias_periodo} dias × 2 períodos)

METODOLOGIA:
• Fonte: Planilha RAW (data/raw)
• Estrutura: Data, Equipe, Logradouro, Período, Quantidade
• Períodos: 10h (Manhã) e 15h (Tarde)
• Extração de Região: Tudo antes do primeiro " - "
• Agrupamento: Por Região e depois por Logradouro
• Ordenação das Regiões: Por soma total de pessoas (decrescente)

CÁLCULOS:
• Soma pessoas: Soma de Quantidade de todos os períodos
• Média pessoas: Soma pessoas ÷ {total_periodos} períodos
• Média por período: Soma do período ÷ {dias_periodo} dias

DADOS PROCESSADOS:
• Registros totais no período: {len(df_periodo):,}
• Regiões encontradas: {total_regioes}
• Logradouros únicos: {total_logradouros}

RESUMO POR REGIÃO:
{resumo_regioes_str}

ARQUIVO GERADO:
✓ {output_file}

ESTRUTURA DAS ABAS:
Para cada região (ordenadas por soma total):
  1. Rank Média [Região] - Todos os logradouros ordenados por média
  2. Abas individuais de cada logradouro (na ordem do rank, com TODAS as datas)

REGIÃO COM MAIOR SOMA TOTAL:
• Região: {regioes_ordenadas[0]}
• Soma total: {soma_por_regiao[regioes_ordenadas[0]]:,.0f} pessoas
• Logradouros: {len(rankings_media[regioes_ordenadas[0]])}
""")

# Top 1 de cada região
print(f"\nTOP 1 DE CADA REGIÃO (POR MÉDIA):")
print("=" * 80)

for regiao in regioes_ordenadas:
    df_regiao = rankings_media[regiao]
    if len(df_regiao) > 0:
        top1 = df_regiao.iloc[0]
        print(f"\n{regiao}:")
        print(f"  • Logradouro: {top1['Logradouro']}")
        print(f"  • Média pessoas: {top1['Média pessoas']:.2f}")
        print(f"  • Manhã: {top1['Manhã']:.2f} | Tarde: {top1['Tarde']:.2f}")

print("\n" + "=" * 80)
print("✓ ANÁLISE CONCLUÍDA!")
print(f"✓ {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
print("=" * 80)

✓ Bibliotecas importadas
✓ Análise iniciada em: 11/11/2025 15:59:02

DEFINIR PERÍODO DO RELATÓRIO

✓ Período selecionado: 01/10/2025 a 31/10/2025

Período: 01/10/2025 a 31/10/2025
  • Dias: 31
  • Períodos por dia: 2 (Manhã e Tarde)
  • Total de períodos: 62

SELEÇÃO DO ARQUIVO RAW
📂 Pasta raw: c:\Users\x504693\Documents\projetos\projeto_etl_dados\data\raw

📁 Arquivos disponíveis (mais recentes primeiro):
  1. Contagem_diaria_centro - Padronizada.xlsx
     Modificado em: 10/11/2025 18:58

  2. Contagem diária - Compilado.xlsx
     Modificado em: 07/11/2025 11:51

  3. CONTAGEM 2025 - CnR.xlsx
     Modificado em: 06/11/2025 12:37

  4. RelatorioCidadaoVinculado.xlsx
     Modificado em: 05/11/2025 16:38

  5. CidadaosVinculadosXBeneficios.xlsx
     Modificado em: 05/11/2025 10:42

  6. Base Okuhara Kohei.xlsx
     Modificado em: 21/10/2025 18:54

  7. nomes_Abordados.xlsx
     Modificado em: 21/10/2025 17:51

  8. Base Tablet.xlsx
     Modificado em: 21/10/2025 17:48

✓ Arquivo seleciona